In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import torch
from torch import optim
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, random_split
from tqdm import tqdm
import torch.optim.lr_scheduler as lr_scheduler

# !pip install torchvision
import torchvision

import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# !pip install torchmetrics
import torchmetrics

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

In [5]:
df = pd.read_csv("/kaggle/input/linear-dataset/linear_dataset.csv")
x, y = df.iloc[:, 0], df.iloc[:, 1]
x = (x - x.min()) / x.max()
y = (y - y.min()) / y.max()

x = np.array(x)
x = x.reshape(len(x), 1) # 化身多行一列的二維數組


model = LinearRegression()
model.fit(x, y)
print(model.score(x, y))

pred = np.array([1, 2])
pred = pred.reshape(2, 1)

ans = model.predict(pred)
print(np.round(ans * (pred.max() - pred.min()) + pred.min()).astype(int)) #反向歸一化及四捨五入, 變回int
#反向歸一化公式 : val * (max - min) + min

0.997599962106742
[2 3]


In [23]:
#neural network version
img, lab = df.iloc[:, 0], df.iloc[:, 1]

img = torch.tensor(img.values)
lab = torch.tensor(lab.values)

img = img.float()
lab = lab.float()
img = (img - img.min()) / img.max()
lab = (lab - lab.min()) / lab.max()

img = img.reshape(len(img), 1) #輸入格式必須為(batch_size, 特徵數)
lab = lab.reshape(len(lab), 1)
dataset = TensorDataset(img, lab)

train_size = int(len(dataset) * 0.8)
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 16, shuffle = True)

In [18]:
class network(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(1, 16)
        self.linear2 = nn.Linear(16, 1)
    def forward(self, x):
        x = self.linear2(self.linear1(x))
        return x
device = torch.device("cpu")
model = network().to(device)
print(model)

network(
  (linear1): Linear(in_features=1, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=1, bias=True)
)


In [28]:
from sklearn.metrics import mean_squared_error
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

scheduler = lr_scheduler.ReduceLROnPlateau( #自動調學習率
    optimizer, 
    mode = 'max', # 監控驗證損失
    factor = 0.1,  # 學習率衰減係數
    patience = 5, # 容忍n個epoch無改善
    verbose = True, # 打印調整日誌
    min_lr = 1e-3 # 最小學習率
)

for i in range(5):
    model.train()
    for (images, labels) in train_loader:
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        pred = outputs.argmax(dim = 1)
        mse = mean_squared_error(labels.numpy(), pred.detach().numpy())
        print(mse)
    model.eval()
    total_loss = 0
    sample = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            total_loss += float(loss_function(outputs, labels)) * img.size(0)
            sample = img.size(0)
        scheduler.step(total_loss / sample)

0.20086845340494958
0.2653461950382702
0.46587647327757453
0.2569844284505217
0.3340870072850836
0.422042505340527
0.42211429892979924
0.22254482355047134
0.24140662054382817
0.23988352052929754
0.23091999107413672
0.40069334192821626
0.4124065537368734
0.3492942860649997
0.3214760539791999
0.3319600892970146
0.3081389532996903
0.25391199622896365
0.23928998499770565
0.18805377511636384
0.2991042832344038
0.28334309326501417
0.43613888774309256
0.3225289861962862
0.3848083122913334
0.3412799239781252
0.2682485089907954
0.3632181425261916
0.3200700471224251
0.35118683287438407
0.33725391837122115
0.19543826448688734
0.35416416150263946
0.31681371539170794
0.29150325066146054
0.3635321637842257
0.3136445443535252
0.2896963708503669
0.39988765833335826
0.4044819847473007
0.3776960310244744
0.3397649182194974
0.23937397969960084
0.2929444536976812
0.3722609822688905
0.3398741820785777
0.2901657248181064
0.37114460745685185
0.3814994147024642
0.35531493066810493
0.31002885213725523
0.357675